<a href="https://colab.research.google.com/github/krMaynard/genAI/blob/main/Chrome_Extension_On_Device_AI_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Please ensure you have imported a Gemini API key from AI Studio.
You can do this directly in the Secrets tab on the left.

After doing so, please run the setup cell below.

In [ ]:
!pip install -U -q "google"
!pip install -U -q "google.genai"

import os
from google.colab import userdata
from google.colab import drive
os.environ["GEMINI_API_KEY"] = userdata.get("GOOGLE_API_KEY")

drive.mount("/content/drive")
# Please ensure that uploaded files are available in the AI Studio folder or change the working folder.
os.chdir("/content/drive/MyDrive/Google AI Studio")

# Code Generation

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types

chrome_extension_prompt = """Code a Chrome extension that uses the built in Chrome LLM API to answer questions about the page and runs in a side panel, using the Chrome Side Panel API."""


def generate(prompt):
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-pro"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=prompt),
            ],
        ),
    ]
    tools = [
        types.Tool(url_context=types.UrlContext()),
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=0,
        thinking_config = types.ThinkingConfig(
            thinking_budget=-1,
        ),
        tools=tools,
        response_mime_type="text/plain",
    )

    # The response is streamed, so we need to accumulate the chunks.
    full_response = []

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        full_response.append(chunk.text)

    # Join all the collected parts to form the complete response.
    return "".join(full_response)

if __name__ == "__main__":
    response = generate(chrome_extension_prompt)
    print (response, end="")


In [14]:
print(response)

This Chrome extension utilizes the built-in Chrome LLM API and the Side Panel API to answer questions about the current web page.

### `manifest.json`

The manifest file defines the extension's core properties, including its name, version, and necessary permissions. It requests access to the `sidePanel` for displaying the UI, `activeTab` and `scripting` to access page content, and the experimental `ai` permission for the LLM functionality.

```json
{
  "manifest_version": 3,
  "name": "Page Q&A Side Panel",
  "version": "1.0",
  "description": "Answers questions about the current page using the built-in Chrome LLM.",
  "permissions": [
    "sidePanel",
    "activeTab",
    "scripting",
    "ai"
  ],
  "side_panel": {
    "default_path": "sidepanel.html"
  },
  "action": {
    "default_title": "Click to open Page Q&A"
  },
  "background": {
    "service_worker": "service-worker.js"
  }
}
```

### `service-worker.js`

This service worker script ensures that the side panel opens when the 